<a href="https://colab.research.google.com/github/Flashcl/pantip_scraper/blob/master/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install pythainlp

     |████████████████████████████████| 11.0MB 211kB/s 
     |████████████████████████████████| 276kB 45.9MB/s 
  Created wheel for marisa-trie: filename=marisa_trie-0.7.5-cp36-cp36m-linux_x86_64.whl size=861347 sha256=506b36cdebce52cd1578c311004d9d40bb286c99914d160adf7f2977387a2ccb
  Stored in directory: /root/.cache/pip/wheels/45/24/79/022624fc914f0e559fe8a1141aaff1f9df810905a13fc75d57
Successfully built marisa-trie


In [0]:
import re
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words

In [0]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

rex = re.compile(r'<div class="display-post-story">(.*?)</div>', re.DOTALL)

def cleanAll(raw_html):
  match = [cleanhtml(i.strip())
            .replace("\t\t","\t").replace("\t"," ")
            .replace("\n\n","\n").replace("\n"," ") 
            for i in rex.findall(data)]
  return match

def cleanWord(t):
  c_t = list()
  for word in t:
    if word.strip():
      c_t.append((word,word in thai_words())) 
  return c_t

In [0]:
data=open("/content/กรุ๊ปเลือด พ่อ B + แม่ B แต่ลูกเราออกมาเป็น AB. - Pantip.htm", "r").read()
test = '<div class="display-post-story">asdfghjkjhgfdsa</div>'

In [0]:
t_list = []
for text in match:
  t = cleanWord(word_tokenize(text))

  t_list.append(t)

In [43]:
t_list[1]

[('พ่อ', True),
 ('แม่', True),
 ('B', False),
 ('ทั้งคู่', True),
 ('ลูก', True),
 ('ต้อง', True),
 ('B', False),
 ('กับ', True),
 ('O', False),
 ('เท่านั้น', True)]